In [ ]:
from scipy.stats.stats import pearsonr
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/Users/desert/desert_workspace/final_data.csv')
df=df.drop('Unnamed: 0', axis=1)
df.head()

In [ ]:
acc=pd.read_excel('/Users/desert/downloads/DataDownload.xls',sheetname=1)
acc

In [ ]:
acc[acc['Variable Code'] == 'FFR12']
acc[acc['Variable Code'] == 'PCH_FFR_07_12']

In [ ]:
acc['Category Code'].unique()
#acc[acc['Category Code']=='PRICES_TAXES']
#acc[acc['Category Code']=='ASSISTANCE']
#acc[acc['Category Code']=='LOCAL']
#acc[acc['Category Code']=='INSECURITY']
#acc[acc['Category Code']=='ACCESS']
#acc[acc['Category Code']=='STORES']
tax=pd.read_excel('/Users/desert/downloads/DataDownload.xls',sheetname=9)
tax=tax[['State','County','MILK_PRICE10','SODA_PRICE10','MILK_SODA_PRICE10']]
tax= tax[tax['State']=='CA']
tax=tax.drop('State',axis=1)
tax.head()
df = pd.merge(df,tax,how='inner',on='County')


In [ ]:
acc_df=pd.read_excel('/Users/desert/downloads/DataDownload.xls',sheetname=2)
acc_df.head()

In [ ]:
#acc_df=pd.read_excel('/Users/desert/downloads/DataDownload.xls',sheetname=9)
acc_df=pd.read_excel('/Users/desert/downloads/DataDownload.xls',sheetname=10)
acc_df = acc_df[acc_df['State'] == 'CA']


In [ ]:
acc_df=acc_df[['County','PCH_FFR_07_12','FFR07','FFR12']]
df=pd.merge(df,acc_df, how='inner',on='County')
df

In [ ]:
df['std_ff']=standardize(df['FFR07'])
sns.lmplot(x='FFR07',y='pop2010_in_des', data=df)

In [ ]:
df.columns.tolist()

In [ ]:
df.columns.tolist()
data = df[['County','des_percent', 'PCH_FFR_07_12','Rural','Urban','cnty_obesity_pct','cnty_dm_pct','cnty_inactive_pct','percent_food_desert', 'pop2010_in_des','POP2010', 'ABR',
 'p_hs_edatt','opiods_rx_1000']]
#data['PCH_FFR_07_12']=data['PCH_FFR_07_12'] * 0.01
data['cnty_obesity_pct']=data['cnty_obesity_pct'] * 0.01
data['cnty_dm_pct']=data['cnty_dm_pct'] * 0.01
data['cnty_inactive_pct']=data['cnty_inactive_pct'] * 0.01
data['p_hs_edatt']=data['p_hs_edatt'] * 0.01
data['ABR']=data['ABR'] * 0.01
data['num_obese'] = np.round(data['cnty_obesity_pct'] * data['POP2010'])
data['opiod_pop']=data['POP2010'] / 1000
#data['opiods_rx_1000'] = data['opiods_rx_1000'] / 1000
data['opiods_rx'] = data['opiods_rx_1000'] * data['opiod_pop']
data['opiods_perc'] = data['opiods_rx'] / data['POP2010']

data.head()

In [ ]:
data['majority_urban'] = data['Urban'] > 0.5
data['majority_urban'] = data['majority_urban'].astype(int)
data['des'] = data['des_percent'] > 0.1
data['des'] = data['des'].astype(int)
# def standardize(variable):
#     return (variable - np.mean(variable)) / np.std(variable)

# data['standardized_pop'] = standardize(data['POP2010'])
# data['standardized_des_pop'] = standardize(data['pop2010_in_des'])
# data['standardized_des_per'] =  standardize(data['pop2010_in_des']) #data['standardized_des_pop'] / data['standardized_pop']
# data.describe()
# data[data['standardized_des_per']>1.259362]

In [ ]:
bayes = data.groupby('majority_urban').agg(np.mean)#[['POP2010','num_obese','cnty_obesity_pct']]
bayes2 = data.groupby('des').agg(np.mean)#[['POP2010','num_obese','cnty_obesity_pct']]

In [ ]:
df.groupby(df['des_percent'] > 0.1).agg(np.mean)[['POP2010','pop2010_in_des','NUMGQTRS','HUNVFlag','PC_PHYS_R','DENTIST_R','PSYCH_R','PCT_HSPNC', 'PCT_BLACK','cnty_dm_pct', 'cnty_obesity_pct','PCT_65OVER','PCT_18_64','Rural','PCT_UNDR18','PCT_UNDER5','PCT_WHITE','unemployment_rate','p_hs_edatt','ABR']]


In [ ]:
#bayes=np.round(bayes)
bayes2

In [ ]:
corrmat = df.corr()

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(16, 12))

# Draw the heatmap using seaborn
sns.heatmap(corrmat, vmax=.8, square=True)
ax.set_title('Data Correlations')
f.tight_layout()

In [ ]:
y = df['pop2010_in_des']
X = df['PCH_FFR_07_12']
linear_regression = smf.ols(formula='pop2010_in_des ~ PCH_FFR_07_12', data=df)
fitted_model = linear_regression.fit()
fitted_model.summary()


In [ ]:
from scipy.stats import chi2_contingency

def chisq_of_df_cols(df, c1, c2):
    contingency_table=pd.crosstab(index=df[c1], columns=df[c2])
    return chi2_contingency(contingency_table, correction=False)

test_stat, pval, deg_freedom, expected = chisq_of_df_cols(df, 'ama', 'readmitted_<30')


test_stat, pval

In [ ]:
from pymc import rbeta


class Discharges(object):

    """
    This class represents N discharge dispositions.

    parameters:
        p_array: a (n,) Numpy array of probabilities >0, <1.

    methods:
        dc( i ): return the results, 0 or 1, of pulling 
                   the ith bandit.
    """

    def __init__(self, p_array):
        self.p = p_array
        self.optimal = np.argmax(p_array)

    def dc(self, i):
        # i is which arm to dc
        return np.random.rand() < self.p[i]

    def __len__(self):
        return len(self.p)


class BayesianStrategy(object):

    """
    Implements a online, learning strategy to solve
    the Multi-Armed Bandit problem.
    
    parameters:
        discharges: a Discharges class with .dc method
    
    methods:
        sample_discharges(n): sample and train on n dc.

    attributes:
        N: the cumulative number of samples
        choices: the historical choices as a (N,) array
        bb_score: the historical score as a (N,) array
    """

    def __init__(self, discharges):

        self.discharges = discharges
        n_discharges = len(self.discharges)
        self.wins = np.zeros(n_discharges)
        self.trials = np.zeros(n_discharges)
        self.N = 0
        self.choices = []
        self.bb_score = []
        
    def sample_discharges(self, n=1):

        bb_score = np.zeros(n)
        choices = np.zeros(n)

        for k in range(n):
            # sample from the discharges's priors, and select the largest sample
            choice = np.argmax(rbeta(1 + self.wins, 1 + self.trials - self.wins))

            # sample the chosen discharge
            result = self.discharges.dc(choice)

            # update priors and score
            self.wins[choice] += result
            self.trials[choice] += 1
            bb_score[k] = result
            self.N += 1
            choices[k] = choice

        self.bb_score = np.r_[self.bb_score, bb_score]
        self.choices = np.r_[self.choices, choices]
        return

In [ ]:
from IPython.core.pylabtools import figsize
import scipy.stats as stats

figsize(11.0, 10)

beta = stats.beta
x = np.linspace(0.001, .999, 200)


def plot_priors(bayesian_strategy, prob, lw=3, alpha=0.2, plt_vlines=True):
    # plotting function
    wins = bayesian_strategy.wins
    trials = bayesian_strategy.trials
    for i in range(prob.shape[0]):
        y = beta(1 + wins[i], 1 + trials[i] - wins[i])
        p = plt.plot(x, y.pdf(x), lw=lw)
        c = p[0].get_markeredgecolor()
        plt.fill_between(x, y.pdf(x), 0, color=c, alpha=alpha,
                         label="underlying probability: %.2f" % prob[i])
        if plt_vlines:
            plt.vlines(prob[i], 0, y.pdf(prob[i]),
                       colors=c, linestyles="--", lw=2)
        plt.autoscale(tight="True")
        plt.title("Posteriors After %d Samples" % bayesian_strategy.N +
                  "s" * (bayesian_strategy.N > 1))
        plt.autoscale(tight=True)
    return

In [ ]:
bayes2#.plot(kind='bar')
bayes2.drop(['pop2010_in_des','POP2010','opiods_rx_1000','num_obese','opiods_rx','opiod_pop'], axis=1).plot(kind='bar')

In [ ]:
true_p_A, true_p_B, N_A, N_B, observations_A, observations_B = gen_obs('cnty_dm_pct','POP2010',bayes2)

# Set up the pymc model. Again assume Uniform priors for p_A and p_B.
p_A = pm.Uniform("p_A", 0, 1)
p_B = pm.Uniform("p_B", 0, 1)


# Define the deterministic delta function. This is our unknown of interest.
@pm.deterministic
def delta(p_A=p_A, p_B=p_B):
    return p_A - p_B

# Set of observations, in this case we have two observation datasets.
obs_A = pm.Bernoulli("obs_A", p_A, value=observations_A, observed=True)
obs_B = pm.Bernoulli("obs_B", p_B, value=observations_B, observed=True)

# To be explained in chapter 3.
mcmc = pm.MCMC([p_A, p_B, delta, obs_A, obs_B])
mcmc.sample(20000, 1000)

In [ ]:
p_A_samples = mcmc.trace("p_A")[:]
p_B_samples = mcmc.trace("p_B")[:]
delta_samples = mcmc.trace("delta")[:]

In [ ]:
def plot_AB(p_A_samples, true_p_A, p_B_samples, true_p_B, delta_samples):
    figsize(12.5, 10)
    ax = plt.subplot(311)
    plt.xlim(0, .1)
    plt.hist(p_A_samples, histtype='stepfilled', bins=25, alpha=0.85,label="posterior of $p_A$", color="#A60628", normed=True)
    plt.vlines(true_p_A, 0, 80, linestyle="--", label="true $p_A$ (unknown)")
    plt.legend(loc="upper right")
    plt.title("Posterior distributions of $p_A$, $p_B$, and delta unknowns")
    ax = plt.subplot(312)
    plt.xlim(0, .1)
    plt.hist(p_B_samples, histtype='stepfilled', bins=25, alpha=0.85,label="posterior of $p_B$", color="#467821", normed=True)
    plt.vlines(true_p_B, 0, 80, linestyle="--", label="true $p_B$ (unknown)")
    plt.legend(loc="upper right")
    ax = plt.subplot(313)
    plt.hist(delta_samples, histtype='stepfilled', bins=30, alpha=0.85,label="posterior of delta", color="#7A68A6", normed=True)
    plt.vlines(true_p_A - true_p_B, 0, 60, linestyle="--",label="true delta (unknown)")
    plt.vlines(0, 0, 60, color="black", alpha=0.2)
    plt.legend(loc="upper right");

plot_AB(p_A_samples, true_p_A, p_B_samples, true_p_B, delta_samples)

In [ ]:
import pymc as pm
figsize(12, 4)

def gen_obs(prob_col,n_col,frame):
    true_p_A = frame[prob_col].ix[0]
    true_p_B = frame[prob_col].ix[1]
    N_A = np.round(frame[n_col].ix[0])
    N_B = np.round(frame[n_col].ix[1])
    observations_A = pm.rbernoulli(true_p_A, N_A)
    observations_B = pm.rbernoulli(true_p_B, N_B)
    print "Obs from Site A: ", observations_A[:30].astype(int), "..."
    print "Obs from Site B: ", observations_B[:30].astype(int), "..."
    return true_p_A, true_p_B, N_A, N_B, observations_A, observations_B
    
true_p_A, true_p_B, N_A, N_B, observations_A, observations_B = gen_obs('cnty_dm_pct','POP2010',bayes)
# # these two quantities are unknown to us.
# true_p_A = bayes['cnty_dm_pct'].ix[0]
# true_p_B = bayes['cnty_dm_pct'].ix[1]

# # notice the unequal sample sizes -- no problem in Bayesian analysis.
# N_A = np.round(bayes['POP2010'].ix[0])
# N_B = np.round(bayes['POP2010'].ix[1])

# # generate some observations
# observations_A = pm.rbernoulli(true_p_A, N_A)
# observations_B = pm.rbernoulli(true_p_B, N_B)
# print "Obs from Site A: ", observations_A[:30].astype(int), "..."
# print "Obs from Site B: ", observations_B[:30].astype(int), "..."

In [ ]:
print observations_A.mean()
print observations_B.mean()

In [ ]:

# Set up the pymc model. Again assume Uniform priors for p_A and p_B.
p_A = pm.Uniform("p_A", 0, 1)
p_B = pm.Uniform("p_B", 0, 1)


# Define the deterministic delta function. This is our unknown of interest.
@pm.deterministic
def delta(p_A=p_A, p_B=p_B):
    return p_A - p_B

# Set of observations, in this case we have two observation datasets.
obs_A = pm.Bernoulli("obs_A", p_A, value=observations_A, observed=True)
obs_B = pm.Bernoulli("obs_B", p_B, value=observations_B, observed=True)

# To be explained in chapter 3.
mcmc = pm.MCMC([p_A, p_B, delta, obs_A, obs_B])
mcmc.sample(20000, 1000)

In [ ]:
p_A_samples = mcmc.trace("p_A")[:]
p_B_samples = mcmc.trace("p_B")[:]
delta_samples = mcmc.trace("delta")[:]

In [ ]:
def plot_AB(p_A_samples, true_p_A, p_B_samples, true_p_B, delta_samples):
    figsize(12.5, 10)
    ax = plt.subplot(311)
    plt.xlim(0, .1)
    plt.hist(p_A_samples, histtype='stepfilled', bins=25, alpha=0.85,label="posterior of $p_A$", color="#A60628", normed=True)
    plt.vlines(true_p_A, 0, 80, linestyle="--", label="true $p_A$ (unknown)")
    plt.legend(loc="upper right")
    plt.title("Posterior distributions of $p_A$, $p_B$, and delta unknowns")
    ax = plt.subplot(312)
    plt.xlim(0, .1)
    plt.hist(p_B_samples, histtype='stepfilled', bins=25, alpha=0.85,label="posterior of $p_B$", color="#467821", normed=True)
    plt.vlines(true_p_B, 0, 80, linestyle="--", label="true $p_B$ (unknown)")
    plt.legend(loc="upper right")
    ax = plt.subplot(313)
    plt.hist(delta_samples, histtype='stepfilled', bins=30, alpha=0.85,label="posterior of delta", color="#7A68A6", normed=True)
    plt.vlines(true_p_A - true_p_B, 0, 60, linestyle="--",label="true delta (unknown)")
    plt.vlines(0, 0, 60, color="black", alpha=0.2)
    plt.legend(loc="upper right");

plot_AB(p_A_samples, true_p_A, p_B_samples, true_p_B, delta_samples)
# ax = plt.subplot(312)


# figsize(12.5, 10)

# # histogram of posteriors

# ax = plt.subplot(311)

# plt.xlim(0, .1)
# plt.hist(p_A_samples, histtype='stepfilled', bins=25, alpha=0.85,
#          label="posterior of $p_A$", color="#A60628", normed=True)
# plt.vlines(true_p_A, 0, 80, linestyle="--", label="true $p_A$ (unknown)")
# plt.legend(loc="upper right")
# plt.title("Posterior distributions of $p_A$, $p_B$, and delta unknowns")

# ax = plt.subplot(312)

# plt.xlim(0, .1)
# plt.hist(p_B_samples, histtype='stepfilled', bins=25, alpha=0.85,
#          label="posterior of $p_B$", color="#467821", normed=True)
# plt.vlines(true_p_B, 0, 80, linestyle="--", label="true $p_B$ (unknown)")
# plt.legend(loc="upper right")

# ax = plt.subplot(313)
# plt.hist(delta_samples, histtype='stepfilled', bins=30, alpha=0.85,
#          label="posterior of delta", color="#7A68A6", normed=True)
# plt.vlines(true_p_A - true_p_B, 0, 60, linestyle="--",
#            label="true delta (unknown)")
# plt.vlines(0, 0, 60, color="black", alpha=0.2)
# plt.legend(loc="upper right");

In [ ]:
hidden_prob = bayes['cnty_obesity_pct'].values
discharges = Discharges(hidden_prob)
bayesian_strat = BayesianStrategy(discharges)

draw_samples = [1, 1, 3, 10, 10, 25, 50, 100, 200, 300]

for j, i in enumerate(draw_samples):
    plt.subplot(5, 2, j + 1)
    bayesian_strat.sample_discharges(i)
    plot_priors(bayesian_strat, hidden_prob)
    plt.legend(labels=['Prob of obesity: Rural','Majority_Urban'])#['home','rehab','LTC','nursing','psych']
    plt.autoscale(tight=True)
plt.tight_layout()

In [ ]:
hidden_prob = bayes['cnty_dm_pct'].values
discharges = Discharges(hidden_prob)
bayesian_strat = BayesianStrategy(discharges)

draw_samples = [1, 1, 3, 10, 10, 25, 50, 100, 200, 300]

for j, i in enumerate(draw_samples):
    plt.subplot(5, 2, j + 1)
    bayesian_strat.sample_discharges(i)
    plot_priors(bayesian_strat, hidden_prob)
    plt.legend(labels=['Majority_Rural','Majority_Urban'])#['home','rehab','LTC','nursing','psych']
    plt.autoscale(tight=True)
plt.tight_layout()

In [ ]:
hidden_prob = bayes['cnty_inactive_pct'].values
discharges = Discharges(hidden_prob)
bayesian_strat = BayesianStrategy(discharges)

draw_samples = [1, 1, 3, 10, 10, 25, 50, 100, 200, 300]

for j, i in enumerate(draw_samples):
    plt.subplot(5, 2, j + 1)
    bayesian_strat.sample_discharges(i)
    plot_priors(bayesian_strat, hidden_prob)
    plt.legend(labels=['Majority_Rural','Majority_Urban'])#['home','rehab','LTC','nursing','psych']
    plt.autoscale(tight=True)
plt.tight_layout()

In [ ]:

hidden_prob = bayes['p_hs_edatt'].values
discharges = Discharges(hidden_prob)
bayesian_strat = BayesianStrategy(discharges)

draw_samples = [1, 1, 3, 10, 10, 25, 50, 100, 200, 300]

for j, i in enumerate(draw_samples):
    plt.subplot(5, 2, j + 1)
    bayesian_strat.sample_discharges(i)
    plot_priors(bayesian_strat, hidden_prob)
    plt.legend(labels=['Majority_Rural','Majority_Urban'])#['home','rehab','LTC','nursing','psych']
    plt.autoscale(tight=True)
plt.tight_layout()